# Shors Algorithm

## Task 1: A classical implementation

First, we need to check special cases for Shor's Algorithm

### 1.1 IsEven

In [1]:
%kata IsEven_Test

operation IsEven(N : Int) : Bool {
    return (N%2==0);
}

Success!

### 1.2 IsPrime

In [2]:
%kata IsPrime_Test

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

operation IsPrime(N : Int) : Bool {
    if (N <= 3) {
        return (N > 1);
    }
    if (N%2 == 0) or (N%3 == 0) {
        return false;
    }
    let sqrtN = Truncate(Sqrt(IntAsDouble(N)));
    for i in 5..6..sqrtN {
        if (N%i == 0) or (N%(i+2) == 0) {
            return false;
        }

    }
    return true;
}

Success!

GCD operation

In [3]:
%kata GreatestCommonDivisor_Test

open Microsoft.Quantum.Math;
operation GreatestCommonDivisor (a : Int, N : Int) : Int {
    let min = Min([a,N]);
    mutable gcd = 1;
    for i in 2..min {
        if (a%i == 0 and N%i == 0) {
            set gcd = i;
        }
    }
    return gcd;
}

Success!

With these two tests, we can confirm that N is a product of two distinct prime numbers, so $N = pq$.
// Explain order finding

### 1.3 Classical order finding

In [39]:
operation FindOrderClassicalHelper(a : BigInt, N: BigInt) : Int {
    mutable power = 0;
    repeat {
        set power += 1;
    } until (a^power % N == 1L);
    return power;
}

FindOrderClassicalHelper

In [41]:
//%kata FindOrderClassical_Test

operation FindOrderClassical(a : Int, N : Int) : Int{
    return FindOrderClassicalHelper(IntAsBigInt(a),IntAsBigInt(N));
}

FindOrderClassical

### 1.4 Generate random number

In [5]:
%kata GenerateRandomNumber_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

operation GenerateRandomNumber(N : Int) : Int {
    use register = Qubit[BitSizeI(N-2)];
    mutable result = 0;
    repeat {
        ApplyToEachCA(H,register);
        set result = MeasureInteger(LittleEndian(register));
    } until ((result + 2) < N);
    return result+2;
}

Success!

### 1.5 General Case

In [8]:

operation GeneralCase(OrderFinder : ((Int, Int)=>Int), N : Int) : (Int, Int) {
    mutable result = 0;
    repeat {
        let a = GenerateRandomNumber(N);
        let gcd = GreatestCommonDivisor(a,N);
        if (gcd > 1) {
            return (gcd, N/gcd);
        }
        let r = OrderFinder(a,N);
        if (IsEven(r)) {
            let x = (a^(r/2) - 1) % N;
            let gcdX = GreatestCommonDivisor(x,N);
            if (gcdX > 1) {
                set result = gcdX;
            }
        }
    } until (result != 0);
    return (result, N/result);
}

GeneralCase

In [14]:
operation GeneralCaseClassical(N : Int) : (Int,Int) {
    GeneralCase(FindOrderClassical,_);
}

/snippet_.qs(2,5): error QS0001: The type Int does not match the type BigInt.
Actual type:   ((Int, Int) => Int)
Expected type: ((BigInt, BigInt) => Int)
/snippet_.qs(2,5): error QS0001: The type Int does not match the type BigInt.
Actual type:   ((Int, Int) => Int)
Expected type: ((BigInt, BigInt) => Int)
/snippet_.qs(2,5): error QS0001: The type (Int => (Int, Int)) does not match the type Unit.
Actual type:   (Int => (Int, Int))
Expected type: Unit
/snippet_.qs(1,1): error QS6307: Not all code paths return a value.


In [13]:
%simulate GeneralCase OrderFinder=FindOrderClassical(_,_) N=4

Received invalid parameters. Please fix and try again:
 OrderFinder: Error converting value "FindOrderClassical(_,_)" to type 'Microsoft.Quantum.Simulation.Core.ICallable'. Path '', line 1, position 25.


### 1.6 Full Classical Shor's Implimentation

In [ ]:
operation ShorsAlgorithmClassical(N : Int) : (Int, Int) {
    
}

In [ ]:
%kata Test_Test

open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;

operation Test(qs : Qubit[]) : Unit {
    ApplyToEach(H,qs);
}

In [ ]:
%kata OrderFindingOracle_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

operation OrderFindingOracle (a : Int, N : Int, power : Int, target : Qubit[]) : Unit is Adj {
    MultiplyByModularInteger(ExpModI(a, power, N), N, LittleEndian(target));
}

In [ ]:
operation OrderFindingTest () : Unit {
    let (a, N, r) = (2, 15, 3);
    use register = Qubit[BitSizeI(N)];
    ApplyToEachCA(H,register);
    OrderFindingOracle(a, N, r, register);
}

In [ ]:
%simulate OrderFindingTest

In [19]:
operation TestErrorCaching() : Unit {
    repeat {
        fail "Something went wrong :(";
    }
    until true
    fixup {
        Message("Yay!");
    }
}

/snippet_.qs(7,9): warning QS6302: This statement will never be executed.


TestErrorCaching

In [18]:
%simulate TestErrorCaching

()